COMP4702 Report
================

Student: Hugo Burton

Date Due: Friday $24^{\text{th}}$ May 2024

In [5]:
import os
import sys
sys.path.append('..')

current_dir = os.getcwd()
print(current_dir)

from welcome import welcome, available_items
from check_log_level import set_log_level

set_log_level()
welcome()


D:\Hugo Burton\Documents\University (2021 - 2024)\2024 Semester 1\COMP4702\COMP4702\Project\report
Log level found LOG_LEVEL=INFO

welcoome
Welcome to the Machine Learning Project
Torch version:  2.2.1+cpu
CUDA is not available
--------------------------------------------------
Python version: 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]
Numpy version: 1.22.4
Seaborn version: 0.13.2
--------------------------------------------------


# Introduction

## Introduction to the dataaset

The ...

### Quantitative Metrics on the dataset



Summary statistics

### Correlelation Matrix



## Objective

- Classify both the **Species** and **Population** of the fruit flies within the dataset using the remaining datapoints.
- Across the three datasets, merge the parameters together then apply the models and rank the attributes in descending order of importance. Additionally, show the top variables in a visual format.
- Find the classification model which performs the best and discuss reasoning for any findings.

# Models

The objective defined above is related to classification of the Loeschcke dataset, meaning only classification models will be used. In this report, the following classifiers are:
- $k$-NN
- Decision Tree > Random Forest
- Neural Network


Each model will be tested equally on the dataset

Techniques will be applied in order to optimise the performance of each model.

## Cross validation setup (maybe)

# $k$-NN

# Decision Tree

# Neural Network



## Ranking of Predictor Variables


# Decision Boundaries

## Certainty of Classification

# Ranking of Classifier Models

The best classifications occur when the certainty is high and the prediction of class is correct. One sample from the
test set was chosen from each classifier for each true class which had maximum certainty and the prediction was
correct. The results are outlined below.

# Worst Errors


# Conclusion

# References

# Appendix